<h1>PRÁCTICA 4</h1>
<h2>Imports y configuración del Notebook</h2>

In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage import io

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Para que no salgan warnings de tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

C:\Users\carlo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<h2>Lectura de datos</h2>

In [2]:
# Archivo donde cada línea contiene "ruta de la imagen" y "número de personas que aparecen en la imagen"
ruta_archivo = 'train.txt'

rutas = []
cantidades = []

# Leer el archivo
with open(ruta_archivo, 'r') as archivo:
    lineas = archivo.readlines()
    # Por cada línea
    for linea in lineas:
        # Separar la línea en partes usando espacios en blanco
        partes = linea.split()  
        # La primera parte es la ruta de la imagen
        ruta = partes[0]
        # La segunda parte es el número de personas que aparecen en la imagen
        cantidad = int(partes[1])
        # Actualizar listas
        rutas.append(ruta)
        cantidades.append(cantidad)


In [3]:
# Leer todas las imágenes
image_collection = io.ImageCollection('./train/train/train/*.png')

# El conjunto de train será la colección de imágenes de las carpetas train
# El conjunto de test serán las cantidades reales obtenidas del archivo anterior
x_train = np.stack(image_collection)
y_train = np.array(cantidades)

In [4]:
# Mostrar el tamaño del conjunto de train (15000 imágenes de 158x158)
x_train.shape

(15000, 158, 158)

<h2>Normalización de los datos</h2>

In [5]:
# En caso de que las imágenes sean en escala de grises, se agregará una nueva dimensión para los canales RGB
x_train = np.expand_dims(x_train, axis = -1)
# Convertir las imágenes a float32
x_train = x_train.astype("float32")

# Obtener el conjunto de train normalizado (Aplicar Z-Score Normalization, donde la media es 0 y la desviación
# estándar es 1)
mean = np.mean(x_train)
std = np.std(x_train)
x_train_normalized = (x_train.copy() - mean) / std

<h2>Creación del modelo de CNN</h2>

In [6]:
## Definición del modelo secuencial Keras
# Definir una capa Input (entrada) de 158x158x1, por lo tanto, las imágenes de entrada serán de 158x158 en escala de grises
# Definir capas Conv2D (convolución) con activación RELU para aprender patrones
# Definir capas max-pooling para reducir el tamaño de las características extraídas
# Añadir capa Flatten (Aplanar) para aplanar las características para prepararlas para la siguiente capa
# Añadir capas Dense (Densa) para la salida del modelo. La última capa solo tiene una unidad ya que solo se espera
# una única predicción numérica.
model = keras.Sequential(
    [
        keras.Input(shape = (158, 158, 1)),
        layers.Conv2D(16, 3, padding = 'valid', activation = 'relu'),
        layers.Conv2D(32, 3, activation = 'relu'),
        layers.MaxPooling2D(pool_size = (2, 2)),
        layers.Conv2D(64, 3, activation = 'relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation = 'relu'),
        layers.Flatten(),
        layers.Dense(64, activation = 'relu'),
        layers.Dense(1),
    ]
)

## Compilación del modelo
# Compilar usando la función de pérdida mean_squared_error para medir la diferencia entre las predicciones y los valores reales
# Utilizar un optimizador Adam con una tasa de aprendizaje de 1e-4
# Agregar RootMeanSquaredError para evaluar el rendimiento durante el entrenamiento
model.compile(
    loss = 'mean_squared_error',
    optimizer = keras.optimizers.Adam(learning_rate =3e-4),
    metrics = [tf.keras.metrics.RootMeanSquaredError()]
)

## Entrenamiento del modelo
# Utilizar un callback ReduceLROnPlateau para reducir la tasa de aprendizaje cuando la pérdida deja de disminuir
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'loss',
    factor = 0.1,
    patience = 1,
    min_lr = 0.000003
)

# Entrenar el modelo usando el conjunto de entrenamiento, usar lotes de 30 muestras, entrenar durante 10 épicas y
# usar el callback creado anteriormente
model.fit(x_train_normalized, y_train, batch_size = 30, epochs = 10, callbacks = [lr_callback])

## Evaluación del modelo
# Evaular en modelo con los mismos datos de entrenamiento y un tamaño de lote de 30
model.evaluate(x_train_normalized, y_train, batch_size = 30)




Epoch 1/10

500/500 [==============================] - 318s 615ms/step - loss: 9.8879 - root_mean_squared_error: 3.1445 - lr: 3.0000e-04
Epoch 2/10
500/500 [==============================] - 291s 583ms/step - loss: 2.7600 - root_mean_squared_error: 1.6613 - lr: 3.0000e-04
Epoch 3/10
500/500 [==============================] - 294s 588ms/step - loss: 2.1403 - root_mean_squared_error: 1.4630 - lr: 3.0000e-04
Epoch 4/10
500/500 [==============================] - 222s 443ms/step - loss: 2.0214 - root_mean_squared_error: 1.4218 - lr: 3.0000e-04
Epoch 5/10
500/500 [==============================] - 213s 427ms/step - loss: 1.6555 - root_mean_squared_error: 1.2867 - lr: 3.0000e-04
Epoch 6/10
500/500 [==============================] - 251s 501ms/step - loss: 1.4817 - root_mean_squared_error: 1.2173 - lr: 3.0000e-04
Epoch 7/10
500/500 [==============================] - 295s 591ms/step - loss: 1.3291 - root_mean_squared_error: 1.1529 - lr: 3.0000e-04
Epoch 8/10
500/500 [=========================

[1.1901334524154663, 1.0909323692321777]

In [7]:
# Mostrar el resumen del modelo
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 156, 156, 16)      160       
                                                                 
 conv2d_1 (Conv2D)           (None, 154, 154, 32)      4640      
                                                                 
 max_pooling2d (MaxPooling2  (None, 77, 77, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 37, 37, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 35, 35, 128)       7

<h2>Predicción del conjunto de test</h2>

In [8]:
# Leer todas las imágenes del test
image_collection_test = io.ImageCollection('./test/test/test/*.png')

# El conjunto X de test será la colección de imágenes de las carpetas test
# En caso de que las imágenes sean en escala de grises, se agregará una nueva dimensión para los canales RGB
x_test = np.stack(image_collection_test)
x_test = np.expand_dims(x_test, axis=-1)

# Obtener el conjunto de test normalizado (Aplicar Z-Score Normalization, donde la media es 0 y la desviación
# estándar es 1)
x_test_normalized = (x_test.copy() - mean) / std

# Predecir los resultados del test usando el modelo entrenado previamente
y_pred = model.predict(x_test_normalized)

16/16 [==============================] - 4s 140ms/step


In [9]:
# Asumiendo que y_pred es tu predicción
df_output = pd.DataFrame(y_pred, columns=['prediction'])
df_output.index.name = 'index'
df_output.to_csv('output/session4/baseline.csv')


Práctica realizada por Adrián Galán Pacheco, Pablo Noriega Vázquez y Carlos Gómez Domínguez.